In [1]:
%run "QS_1.ipynb"
transaction['DATE'] = pd.to_datetime(transaction['DATE'], origin='1899-12-30', unit='D')
transaction['year_mm']=transaction['DATE'].dt.strftime('%Y%m').astype(int)
ts=transaction.groupby(['year_mm','STORE_NBR'])['TOT_SALES'].sum().reset_index()
NoC=transaction.groupby(['year_mm','STORE_NBR'])['LYLTY_CARD_NBR'].nunique().reset_index()
TxNo=transaction.groupby(['year_mm','STORE_NBR'])['TXN_ID'].nunique().reset_index()
Pq=transaction.groupby(['year_mm','STORE_NBR'])['PROD_QTY'].sum().reset_index()
matrix=pd.merge(ts,NoC,on=['year_mm','STORE_NBR'])
matrix=pd.merge(TxNo,matrix,on=['year_mm','STORE_NBR'])
matrix=pd.merge(Pq,matrix,on=['year_mm','STORE_NBR'])
matrix.rename(columns={
    'STORE_NBR':'Store Number',
    'PROD_QTY':'Total Chips',
    'TXN_ID':' Number of Transactions',
    'LYLTY_CARD_NBR':'Number of Customers',
    'TOT_SALES':'Total Sales'},inplace=True)
def uplift(trial,control):
    Trial_pre=matrix[(matrix['Store Number']==trial)&(matrix['year_mm']<201902)]
    Trial_post=matrix[(matrix['Store Number']==trial)&(matrix['year_mm']>201902)]
    control_pre=matrix[(matrix['Store Number']==control)&(matrix['year_mm']<201902)]
    control_post=matrix[(matrix['Store Number']==control)&(matrix['year_mm']>201902)]
#grouping by months and sum
    Trial_gr=Trial_pre.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
    control_gr=control_pre.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
    Trial_go=Trial_post.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
    control_go=control_post.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
#merging
    pre=pd.merge(Trial_gr,control_gr,on='year_mm')
    post=pd.merge(Trial_go,control_go,on='year_mm')
#ratios
    pre['ratio_ts']=pre['Total Sales_x']/pre['Total Sales_y']
    pre['ratio_Noc']=pre['Number of Customers_x']/pre['Number of Customers_y']
    post['ratio_ts']=post['Total Sales_x']/post['Total Sales_y']
    post['ratio_Noc']=post['Number of Customers_x']/post['Number of Customers_y']
###averages
    pre_avg_ts=pre['ratio_ts'].mean()
    pre_avg_Noc=pre['ratio_Noc'].mean()
    post_avg_ts=post['ratio_ts'].mean()
    post_avg_Noc=post['ratio_Noc'].mean()
#Calculating uplifting
    uplift_ts=(post_avg_ts-pre_avg_ts)/pre_avg_ts*100
    uplift_Noc=(post_avg_Noc-pre_avg_Noc)/pre_avg_Noc*100
    return {
        'Trial Store': trial,
        'Control Store': control,
        'Uplift_Sales (%)': round(uplift_ts, 2),
        'Uplift_Customers (%)': round(uplift_Noc, 2)
    }
trial = [77, 86, 88]
control = [233, 155, 237]
results = []
for trial, control in zip(trial, control):
    results.append(uplift(trial, control))
# Converting the outcome to DataFrame
Summary_uplifting = pd.DataFrame(results)
print (Summary_uplifting)

   Trial Store  Control Store  Uplift_Sales (%)  Uplift_Customers (%)
0           77            233             23.46                 18.86
1           86            155              8.65                  7.39
2           88            237             16.62                  5.60
